In [1]:
import os
from typing import List, Tuple
import wfdb
from wfdb import processing
from tqdm import tqdm
import numpy as np


def clean_annotations(annotation: wfdb.Annotation) -> Tuple[List[int], List[str]]:
    """
    Remove anotações vazias da lista de anotações.

    Args:
        annotation (wfdb.Annotation): Anotação contendo as amostras e notas auxiliares.

    Returns:
        Tuple[List[int], List[str]]: Listas de amostras e notas auxiliares não vazias.
    """
    sample, aux_note = annotation.sample, annotation.aux_note
    non_empty_indices = [i for i, note in enumerate(aux_note) if note != ""]
    clean_aux_note = [aux_note[i] for i in non_empty_indices]
    clean_sample = [sample[i] for i in non_empty_indices]
    return clean_sample, clean_aux_note

def get_ranges_afib(record_path: str, signal_len: int) -> List[Tuple[int, int]]:
    """
    Obtém os intervalos de interesse onde a anotação é AFIB.

    Args:
        record_path (str): Caminho para o registro do sinal.
        signal_len (int): Comprimento total do sinal.

    Returns:
        List[Tuple[int, int]]: Lista de tuplas com os intervalos de início e fim onde a anotação é AFIB.
    """
    annotation = wfdb.rdann(record_path, "atr")
    sample, aux_note = clean_annotations(annotation)
    ranges_interest = []

    for i, label in enumerate(aux_note):
        if label == "(AFIB":
            afib_start = sample[i]
            afib_end = signal_len if i == len(sample) - 1 else sample[i + 1] - 1
            ranges_interest.append((afib_start, afib_end))

    return ranges_interest


In [2]:
def process_records(database_path: str) -> None:
    """
    Processa todos os registros no caminho especificado, excluindo os registros indesejados.

    Args:
        database_path (str): Caminho para o diretório contendo os registros.
    """
    # Carregar os IDs dos registros
    record_ids = []
    with open(os.path.join(database_path, "RECORDS")) as f:
        record_ids = [line.strip() for line in f.readlines()]

    # Remover registros indesejados da base AFDB
    record_ids.remove("00735")
    record_ids.remove("03665")

    # Processar cada registro
    for record_index, record_id in enumerate(record_ids):
        record_path = os.path.join(database_path, record_id)
        _, ecg_metadata = wfdb.rdsamp(record_path)
        signal_len = ecg_metadata["sig_len"]
        
        # Gravação completa
        recording = wfdb.rdrecord(record_name=record_path)

        extract_intervals = get_ranges_afib(record_path, signal_len)
        
        # Para teste, processa apenas o primeiro registro
        print(f"Record ID: {record_id}, AFIB Intervals: {extract_intervals}")

        # Quantidade de RRIs em cada segmento do sinal ECG
        SEGMENT_SIZE = 5 
        
        stack_rr = np.empty((0, SEGMENT_SIZE), dtype=int)
        stack_recording = []
        
        # Percorre cada trecho diagnosticado com AFIB
        for start_index, end_index in tqdm(extract_intervals):
            annotations_r_peaks = wfdb.rdann(
                record_path,
                sampfrom=start_index,
                sampto=end_index,
                extension="qrs",
            )
            positions_r_peaks = annotations_r_peaks.sample
            frequency = annotations_r_peaks.fs
            
            positions_r_peak_ms = (positions_r_peaks / frequency) * 1000
            
            rr_intervals = processing.calc_rr(positions_r_peak_ms, fs=frequency)
            
            num_segments = (len(positions_r_peaks) - 1) // SEGMENT_SIZE

            if num_segments <= 0:
                continue

            last_segment = num_segments * SEGMENT_SIZE

            for i in range(0, last_segment, SEGMENT_SIZE):
                # Montando a saída dos segmentos (RRIs)
                rr_segment = rr_intervals[i : i + SEGMENT_SIZE]
                stack_rr = np.vstack((stack_rr, rr_segment))

                # Montando a saída dos segmentos (Gravação 2 derivações)
                start_index = positions_r_peaks[i]
                end_index = positions_r_peaks[i + SEGMENT_SIZE]
                rec_seg = recording.p_signal[start_index:end_index]
                stack_recording.append(rec_seg)
        
        destination_results = 'exploring_afdb_results'

        if not os.path.exists(destination_results):
            os.makedirs(destination_results)
        
        np.save(
            file=f"./exploring_afdb_results/{record_index}_{record_id}_rri_segment", arr=stack_rr
        )

        stack_recording = np.array(stack_recording, dtype=object)
        
        np.save(
            file=f"./exploring_afdb_results/{record_index}_{record_id}_recording_segment",
            arr=stack_recording,
        )
        
        # Esse break serve para rodar apenas para a primeira gravação (Teste)
        break

DATABASE_PATH = './../data/mit-bih-atrial-fibrillation-database-1.0.0/files/'
process_records(DATABASE_PATH)

Record ID: 04015, AFIB Intervals: [(102584, 119603), (121773, 122193), (133348, 166856), (1096245, 1098053), (1135296, 1139594), (1422436, 1423547), (1459277, 1460415)]


100%|██████████| 7/7 [00:00<00:00, 28.63it/s]


In [3]:
# Trechos da gravação 04015 particionadas em segmentos de 5 RRIs que foram diagnosticadas com FA

np.load(f"./exploring_afdb_results/0_04015_recording_segment.npy", allow_pickle=True)

array([array([[-1.33 , -0.18 ],
              [-1.155, -0.195],
              [-1.04 , -0.135],
              ...,
              [-0.865, -1.46 ],
              [-0.98 , -1.74 ],
              [-0.875, -1.9  ]]), array([[-0.755, -2.06 ],
                                         [-0.505, -2.145],
                                         [-0.19 , -2.055],
                                         ...,
                                         [-1.29 , -1.535],
                                         [-1.63 , -1.875],
                                         [-1.57 , -2.13 ]]),
       array([[-1.38 , -2.335],
              [-1.065, -2.39 ],
              [-0.745, -2.29 ],
              [-0.48 , -1.965],
              [-0.355, -1.54 ],
              [-0.27 , -1.155],
              [-0.225, -0.835],
              [-0.21 , -0.585],
              [-0.215, -0.465],
              [-0.205, -0.335],
              [-0.205, -0.27 ],
              [-0.205, -0.225],
              [-0.18 , -0.21 ],
   

In [4]:
# Os RRIs da gravação 04015 agrupados em segmentos de tamanho 5 diagnosticados com FA

np.load(f"./exploring_afdb_results/0_04015_rri_segment.npy", allow_pickle=True)

array([[552., 492., 416., 396., 436.],
       [532., 568., 424., 296., 384.],
       [328., 364., 388., 360., 296.],
       [340., 376., 364., 372., 492.],
       [428., 572., 448., 632., 472.],
       [464., 420., 392., 400., 484.],
       [428., 316., 652., 508., 356.],
       [440., 500., 384., 520., 448.],
       [372., 492., 424., 572., 524.],
       [364., 428., 392., 288., 628.],
       [504., 528., 504., 528., 472.],
       [348., 448., 524., 404., 476.],
       [460., 516., 432., 460., 416.],
       [492., 544., 456., 320., 316.],
       [428., 536., 500., 380., 372.],
       [472., 440., 300., 384., 448.],
       [424., 352., 420., 396., 380.],
       [416., 388., 376., 348., 328.],
       [356., 376., 460., 560., 392.],
       [304., 380., 488., 452., 556.],
       [396., 368., 704., 680., 684.],
       [860., 468., 444., 452., 316.],
       [328., 436., 452., 384., 312.],
       [608., 564., 428., 528., 396.],
       [344., 404., 392., 324., 428.],
       [672., 500., 332.,